<a href="https://colab.research.google.com/github/gafm-ai/My_Colab_Notebook/blob/main/I_mod1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try :
    import finnhub
except :
    !pip install finnhub-python
    import finnhub

import pandas as pd
from datetime import datetime, timedelta
import time
import os
from requests.exceptions import ReadTimeout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# Configuration
FINNHUB_API_KEY = 'cpq6j4pr01qo647ne0qgcpq6j4pr01qo647ne0r0'  # Remplacez par votre clé API Finnhub

# Initialiser le client Finnhub
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

In [ ]:
import os
import time
import logging
import finnhub
import pandas as pd
from datetime import datetime
from requests.exceptions import ReadTimeout, ConnectionError

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def convert_timeframe_to_minutes(timeframe):
    """
    Convertit un timeframe en minutes.

    Args:
        timeframe (str or int): La période de temps (ex: "5", "15", "D", "W", "M").

    Returns:
        int: Le timeframe en minutes.

    Raises:
        ValueError: Si le timeframe n'est pas valide.
    """
    if isinstance(timeframe, int) or (isinstance(timeframe, str) and timeframe.isdigit()):
        return int(timeframe)
    elif timeframe == 'D':
        return 24 * 60
    elif timeframe == 'W':
        return 7 * 24 * 60
    elif timeframe == 'M':
        return 30 * 24 * 60
    else:
        raise ValueError(f"Timeframe non valide : {timeframe}")

def fetch_price(symbol, timeframe, start_timestamp, end_timestamp, instrument_type='forex'):
    """
    Récupère les données de prix depuis l'API Finnhub pour un symbole donné.

    Args:
        symbol (str): Le symbole de l'actif.
        timeframe (str or int): La période de temps.
        start_timestamp (int): Timestamp Unix de début.
        end_timestamp (int): Timestamp Unix de fin.
        instrument_type (str, optional): Type d'instrument ('forex', 'stock', 'crypto'). Par défaut 'forex'.

    Returns:
        pd.DataFrame: Données de prix avec les colonnes ['timestamp', 'open', 'high', 'low', 'close', 'volume'].
    """
    timeframe_minutes = convert_timeframe_to_minutes(timeframe)
    all_data_price = []
    current_start = start_timestamp

    while current_start < end_timestamp:
        try:
            # Calcul de la fin actuelle en respectant la limite de 2000 bougies par requête
            current_end = min(current_start + 2000 * timeframe_minutes * 60, end_timestamp)

            # Sélection de la fonction API appropriée en fonction du type d'instrument
            if instrument_type == 'forex':
                price_data = finnhub_client.forex_candles(symbol, timeframe, current_start, current_end)
            elif instrument_type == 'stock':
                price_data = finnhub_client.stock_candles(symbol, timeframe, current_start, current_end)
            elif instrument_type == 'crypto':
                price_data = finnhub_client.crypto_candles(symbol, timeframe, current_start, current_end)
            else:
                raise ValueError(f"Type d'instrument non supporté : {instrument_type}")

            # Vérification du statut de la réponse
            if price_data['s'] == 'no_data':
                logging.warning(f"Aucune donnée disponible pour {symbol} entre {datetime.fromtimestamp(current_start)} et {datetime.fromtimestamp(current_end)}.")
                current_start = current_end + 1
                continue  # Continuer l'exécution sans ajouter de données

            if price_data['s'] != 'ok':
                logging.error(f"Erreur API Finnhub pour {symbol}: {price_data.get('s')}")
                current_start = current_end + 1
                continue  # Continuer l'exécution malgré l'erreur

            # Conversion des données en DataFrame
            price_df = pd.DataFrame(price_data)
            all_data_price.append(price_df)

            current_start = current_end + 1
            time.sleep(0.5)  # Pause pour respecter les limites de taux

        except (ReadTimeout, ConnectionError) as e:
            logging.warning(f"Problème réseau lors de la récupération des données pour {symbol}: {e}. Nouvelle tentative dans 5 secondes...")
            time.sleep(5)
        except Exception as e:
            logging.error(f"Erreur inattendue: {e}")
            break  # Arrêter la boucle en cas d'erreur inattendue

    if not all_data_price:
        logging.warning("Aucune donnée n'a été récupérée.")
        return pd.DataFrame()  # Retourne un DataFrame vide

    # Concaténation de toutes les données récupérées
    final_df_price = pd.concat(all_data_price, ignore_index=True)
    final_df_price['timestamp'] = pd.to_datetime(final_df_price['t'], unit='s', utc=True)
    final_df_price.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}, inplace=True)
    final_df_price.sort_values('timestamp', inplace=True)

    return final_df_price[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

def fetch_and_store_price_data(symbol, timeframe, start_date, end_date, output_directory, instrument_type='forex'):
    """
    Récupère et enregistre les données de prix de Finnhub pour un symbole donné.

    Args:
        symbol (str): Le symbole de l'actif (ex: "OANDA:EUR_USD").
        timeframe (str or int): La période de temps (ex: "5", "15", "D", "W", "M").
        start_date (datetime): La date de début.
        end_date (datetime): La date de fin.
        output_directory (str): Le chemin vers le répertoire de sortie.
        instrument_type (str, optional): Le type d'instrument ('forex', 'stock', 'crypto'). Par défaut 'forex'.
    """
    if start_date >= end_date:
        logging.error("La date de début doit être antérieure à la date de fin.")
        return

    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())

    # Récupération des données de prix
    price_data = fetch_price(symbol, timeframe, start_timestamp, end_timestamp, instrument_type)

    if price_data.empty:
        logging.warning(f"Aucune donnée à enregistrer pour {symbol}.")
        return

    # Création du répertoire de sortie si nécessaire
    os.makedirs(output_directory, exist_ok=True)

    # Définition du nom du fichier de sortie
    symbol_safe = symbol.replace(":", "_").replace("/", "_")
    file_name = f'{symbol_safe}_{timeframe}_{start_date.strftime("%Y%m%d")}_{end_date.strftime("%Y%m%d")}_price.csv'
    file_path = os.path.join(output_directory, file_name)

    # Enregistrement des données au format CSV
    price_data.to_csv(file_path, index=False)
    logging.info(f"Fichier enregistré localement : {file_path}")

# Exemple d'utilisation de la fonction
if __name__ == "__main__":
    # Définition des paramètres
    symbole = "OANDA:EUR_USD"  # Exemple pour le forex
    timeframe = "5"  # 60 minutes
    start_date = datetime(2024, 5, 1)
    end_date = datetime(2024, 10, 5)
    repertoire_sortie = "/content/drive/MyDrive/FLEURY/Finances/test_data"
    type_instrument = "forex"

    # Appel de la fonction pour récupérer et enregistrer les données
    fetch_and_store_price_data(symbole, timeframe, start_date, end_date, repertoire_sortie, type_instrument)


In [ ]:
cc

In [ ]:
price_data = finnhub_client.forex_candles("CAPITAL:EURUSD", '5', int(datetime(2023, 1, 1).timestamp()), int(datetime(2023, 1, 5).timestamp()))

In [ ]:
# prompt: prend la donnée des 20 derniere minute depuis finnhub

from datetime import datetime, timedelta

now = datetime.now()
twenty_minutes_ago = now - timedelta(minutes=20)

price_data = finnhub_client.forex_candles("OANDA:EUR_USD", '5', int(twenty_minutes_ago.timestamp()), int(now.timestamp()))

if price_data['s'] == 'ok':
  df = pd.DataFrame(price_data)
  df['timestamp'] = pd.to_datetime(df['t'], unit='s', utc=True)
  df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}, inplace=True)
  print(df[['timestamp', 'open', 'high', 'low', 'close', 'volume']])
else:
  print("No data available for the last 20 minutes")


In [ ]:
2024-10-08 12:05:00+00:00  1.09877  1.09882  1.09875  1.09879      44

In [ ]:
price_data.keys()

In [ ]:
price_data

In [ ]:
avc

In [ ]:
# Exécution avec des paramètres personnalisés
symbol = 'OANDA:EUR_USD'  # Par exemple, EUR/USD
timeframe = '5'  # Timeframe de 5 minutes
start_date = datetime(2012, 1, 1)
end_date = datetime(2022, 12, 31)
output_directory = os.path.expanduser('/content/drive/MyDrive/FLEURY/Finances/Finances_data')  # Spécifier le répertoire de sortie

# Appel de la fonction avec les paramètres définis
fetch_and_store_price_data(symbol, timeframe, start_date, end_date, output_directory)

In [ ]:
# Fonction pour récupérer les données de prix de Finnhub
import finnhub
import pandas as pd
from datetime import datetime, timedelta
import time
import os
from requests.exceptions import ReadTimeout


# Initialiser le client Finnhub
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)


def convert_timeframe_to_minutes(timeframe):
    if isinstance(timeframe, int) or timeframe.isdigit():
        return int(timeframe)
    elif timeframe == 'D':
        return 24 * 60
    elif timeframe == 'W':
        return 7 * 24 * 60
    elif timeframe == 'M':
        return 30 * 24 * 60
    else:
        raise ValueError(f"Timeframe non valide : {timeframe}")

def fetch_price(symbol, timeframe, start_timestamp, end_timestamp, instrument_type='forex'):
    timeframe_minutes = convert_timeframe_to_minutes(timeframe)

    all_data_price = []
    current_start = start_timestamp
    request_count = 0
    start_time = time.time()

    while current_start < end_timestamp:
        try:
            if request_count >= 150:
                elapsed_time = time.time() - start_time
                if elapsed_time < 60:
                    time.sleep(60 - elapsed_time)
                request_count = 0
                start_time = time.time()

            current_end = min(current_start + 2000 * timeframe_minutes * 60, end_timestamp)

            if instrument_type == 'forex':
                price_data = finnhub_client.forex_candles(symbol, timeframe, current_start, current_end)
            elif instrument_type == 'stock':
                price_data = finnhub_client.stock_candles(symbol, timeframe, current_start, current_end)
            elif instrument_type == 'crypto':
                price_data = finnhub_client.crypto_candles(symbol, timeframe, current_start, current_end)
            else:
                raise ValueError(f"Type d'instrument non supporté : {instrument_type}")

            request_count += 1

            if price_data['s'] != 'ok':
                raise Exception('Erreur lors de la récupération des données depuis Finnhub')

            price_df = pd.DataFrame(price_data)
            all_data_price.append(price_df)

            current_start = current_end + 1
            time.sleep(0.5)
        except ReadTimeout:
            print(f"Timeout lors de la récupération des données pour {symbol}. Nouvelle tentative...")
            time.sleep(5)
            continue

    if not all_data_price:
        raise Exception("Aucune donnée n'a été récupérée")

    final_df_price = pd.concat(all_data_price, ignore_index=True)
    final_df_price['timestamp'] = pd.to_datetime(final_df_price['t'], unit='s')
    final_df_price.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}, inplace=True)

    return final_df_price[['timestamp', 'open', 'high', 'low', 'close', 'volume']]



def fetch_and_store_price_data(symbol, timeframe, start_date, end_date, output_directory, instrument_type='forex'):
    """
    Récupère et enregistre les données de prix de Finnhub pour un symbole donné.

    Args:
        symbol (str): Le symbole de l'actif (ex: "EURUSD").
        timeframe (str): La période de temps (ex: "5", "15", "D", "W", "M").
        start_date (datetime): La date de début.
        end_date (datetime): La date de fin.
        output_directory (str): Le chemin vers le répertoire de sortie.
        instrument_type (str, optional): Le type d'instrument (ex: "forex", "stock", "crypto"). Defaults to "forex".
    """
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())

    # Récupérer les données de prix de Finnhub
    price_data = fetch_price(symbol, timeframe, start_timestamp, end_timestamp, instrument_type)

    # Créer le répertoire de sortie si nécessaire
    os.makedirs(output_directory, exist_ok=True)

    # Définir le nom du fichier de sortie
    file_name = f'{symbol.replace(":", "_")}_{timeframe}_{start_date.strftime("%Y%m%d")}_{end_date.strftime("%Y%m%d")}_price.csv'
    file_path = os.path.join(output_directory, file_name)

    # Enregistrer les données de prix au format CSV
    price_data.to_csv(file_path, index=False)
    print(f"Fichier enregistré localement : {file_path}")







def fetch_indicator_data(symbol, indicator, timeframe, start_timestamp, end_timestamp, keep_ohlcv=False, **kwargs):
    timeframe_minutes = convert_timeframe_to_minutes(timeframe)

    # Calculer le décalage maximum basé sur les paramètres de période
    max_period = max([int(v) for k, v in kwargs.items() if 'period' in k.lower()] or [0])
    offset = (max_period + 1) * timeframe_minutes * 60  # Convertir en secondes

    max_retries = 3
    retry_delay = 5
    all_data_indicator = []
    current_start = start_timestamp
    request_count = 0
    start_time = time.time()

    while current_start < end_timestamp:
        for attempt in range(max_retries):
            try:
                if request_count >= 150:
                    elapsed_time = time.time() - start_time
                    if elapsed_time < 60:
                        time.sleep(60 - elapsed_time)
                    request_count = 0
                    start_time = time.time()

                request_start = max(current_start - offset, 0)  # Appliquer l'offset pour chaque requête
                current_end = min(current_start + 2000 * timeframe_minutes * 60, end_timestamp)

                indicator_data = finnhub_client.technical_indicator(
                    symbol=symbol,
                    resolution=timeframe,
                    _from=request_start,
                    to=current_end,
                    indicator=indicator,
                    indicator_fields=kwargs
                )

                request_count += 1

                if not indicator_data or 's' not in indicator_data or indicator_data['s'] != 'ok':
                    raise Exception(f'Aucune donnée retournée pour {indicator}.')

                indicator_df = pd.DataFrame(indicator_data)
                indicator_df['timestamp'] = pd.to_datetime(indicator_df['t'], unit='s')

                # Garder seulement les données à partir de current_start
                indicator_df = indicator_df[indicator_df['timestamp'] >= pd.to_datetime(current_start, unit='s')]

                all_data_indicator.append(indicator_df)

                current_start = current_end + 1
                time.sleep(0.5)
                break
            except Exception as e:
                print(f"Erreur lors de la récupération de {indicator} : {e}")
                if attempt == max_retries - 1:
                    return None
                time.sleep(retry_delay)

    if not all_data_indicator:
        return None

    final_df_indicator = pd.concat(all_data_indicator, ignore_index=True)
    columns_to_drop = ['t', 's']
    if not keep_ohlcv:
        columns_to_drop.extend(['c', 'h', 'l', 'o', 'v'])
    final_df_indicator = final_df_indicator.drop(columns=columns_to_drop)

    return final_df_indicator

# Fonction améliorée pour récupérer tous les indicateurs nécessaires
def fetch_all_indicators(symbol, resolution, start_timestamp, end_timestamp):
    indicators_params = {
        'kama': {'indicator': 'kama', 'timeperiod': 10},
        'stochrsi': {'indicator': 'stochrsi', 'timeperiod': 14},
        'aroon': {'indicator': 'aroon', 'timeperiod': 14},
        'sar': {'indicator': 'sar', 'acceleration': 0.02, 'maximum': 0.2},
        'atr': {'indicator': 'atr', 'timeperiod': 14},
        'adosc': {'indicator': 'adosc', 'fastperiod': 6, 'slowperiod': 14},
        'httrendmode': {'indicator': 'httrendmode'},
        'ema50': {'indicator': 'ema', 'timeperiod': 50},
    }

    all_data = {}
    for indicator, params in indicators_params.items():
        indicator_name = params.pop('indicator')
        data = fetch_indicator_data(symbol, indicator_name, resolution, start_timestamp, end_timestamp, **params)
        if data is not None:
            # Assurez-vous que la colonne de temps est nommée 'timestamp'
            if 't' in data.columns:
                data = data.rename(columns={'t': 'timestamp'})
            if 'timestamp' in data.columns:
                all_data[indicator] = data
            else:
                print(f"Avertissement : Pas de colonne 'timestamp' pour l'indicateur {indicator}")
        time.sleep(1)  # Pause pour éviter les limitations de l'API

    return all_data

# Fonction principale mise à jour
def fetch_and_store_data(symbol, timeframe, start_date, end_date, output_directory, instrument_type='forex'):
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())

    # Récupérer les données de prix de Finnhub
    price_data = fetch_price(symbol, timeframe, start_timestamp, end_timestamp, instrument_type)

    # Récupérer les indicateurs techniques
    indicators_data = fetch_all_indicators(symbol, timeframe, start_timestamp, end_timestamp)

    # Fusionner les données de prix avec les indicateurs
    for name, data in indicators_data.items():
        if 'timestamp' in data.columns:
            price_data = price_data.merge(data, on='timestamp', how='left')
        else:
            print(f"Impossible de fusionner les données pour l'indicateur {name} : pas de colonne 'timestamp'")

    # Créer le répertoire de sortie si nécessaire
    os.makedirs(output_directory, exist_ok=True)

    # Définir le nom du fichier de sortie
    file_name = f'{symbol.replace(":", "_")}_{timeframe}_{start_date.strftime("%Y%m%d")}_{end_date.strftime("%Y%m%d")}_with_indicators.csv'
    file_path = os.path.join(output_directory, file_name)

    # Enregistrer les données fusionnées au format CSV
    price_data.to_csv(file_path, index=False)
    print(f"Fichier enregistré localement : {file_path}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Exécution avec des paramètres personnalisés
symbol = 'OANDA:EUR_USD'  # Par exemple, EUR/USD
timeframe = '5'  # Timeframe de 5 minutes
start_date = datetime(2012, 1, 1)
end_date = datetime(2022, 12, 31)
output_directory = os.path.expanduser('/content/drive/MyDrive/FLEURY/Finances/Finances_data')  # Spécifier le répertoire de sortie

# Appel de la fonction avec les paramètres définis
fetch_and_store_price_data(symbol, timeframe, start_date, end_date, output_directory)

In [ ]:
# prompt: lire le fichier et affiche infos

import pandas as pd

# Spécifier le chemin du fichier
file_path = '/content/drive/MyDrive/FLEURY/Finances/Finances_data/OANDA_EUR_USD_5_20120101_20221231_price.csv'

# Lire le fichier CSV
df = pd.read_csv(file_path)

# Afficher les premières lignes du DataFrame
print(df.head())

# Afficher des informations sur le DataFrame
print(df.info())

# Afficher des statistiques descriptives
print(df.describe())
